In [ ]:
from imports_scgen_local import * #imports_scgen_local have to be in same folder

In [ ]:
pre = "MH65"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

In [ ]:
celltypes = ['B',
 'CD14 Mono',
 #'CD16 Mono',
 'CD4 T',
 'CD8 T',
 'MAIT',
 'NK+Proliferating']

In [ ]:
model_path = os.path.join(*["F:\\","monkey_IZI","analysis","cluster_runs","workspace-runVAE",
                            "output","MH49_c","models"])


In [ ]:
all_species = ['cyno','human']
all_conditions = ['00hr','06hr','24hr']
n_latent = 10
shifts_within_species = ['00hr->06hr','00hr->24hr','06hr->24hr']


In [ ]:
warnings.filterwarnings("ignore")
adata_comps_06hr = {}
r2_dicts_06hr = {}
mean_exp_dicts_06hr = {}
gene_names_dict_06hr = {}
model_dict_06hr = {}
for model_dir_name in celltypes:
    load_string = os.path.join(model_path,model_dir_name,'MH49_c_scgen_model_' + str(model_dir_name)+'.pt')
    model = scgen.SCGEN.load(load_string)
    model.adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    model.adata.obs = h.make_annotation_azimut1_5_scanvi_v2(model.adata)
    #if model_dir_name == 'CD4 T':
    #    model.adata = model.adata[(model.adata.obs['scvi_clusters'] != 41)&(model.adata.obs['scvi_clusters'] != 53)]
    model_dict_06hr[model_dir_name] = model
    adata_comp,r2_dict,mean_exp_dict = hscg.similarity_GE_after_shift_06hr(model=model,
                                   adata_train=model.adata,
                                   adata_test=model.adata,
                                   celltype_col= 'cluster_azimut1_5_scanvi_v2',
                                   celltype=model_dir_name)
    adata_comps_06hr[model_dir_name] = adata_comp
    r2_dicts_06hr[model_dir_name] = r2_dict
    mean_exp_dicts_06hr[model_dir_name] = mean_exp_dict
    gene_names_dict_06hr[model_dir_name] = model.adata.var.index.values

In [ ]:
df_test_06hr = pd.DataFrame(index = ['rank_00hr_human', 'rank_06hr_human', 'rank_24hr_human'])
for celltype in celltypes:
    df_ct = hscg.make_rank_df_06hr(mean_exp_dicts_06hr,adata_comps_06hr,celltype=celltype)
    df_test_06hr=df_test_06hr.join(df_ct)

In [ ]:
df_test_06hr.index = ['00hr', '06hr', '24hr']

plt.figure()
sns.heatmap(df_test_06hr.T, annot=True, cmap="PuBuGn", 
            cbar_kws={'label': 'Spearman rank correlation \n predicted human 06hr from cyno 06hr'},
            norm=Normalize(vmin=0.4, vmax=1),
            linecolor='grey',
            linewidths=0.05)
plt.yticks(rotation=0)
plt.savefig(os.path.join(base_plots_path, pre + '_heatmap_rank_06hr.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
warnings.filterwarnings("ignore")
adata_comps_24hr = {}
r2_dicts_24hr = {}
mean_exp_dicts_24hr = {}
gene_names_dict_24hr = {}
model_dict_24hr = {}
for model_dir_name in celltypes:
    load_string = os.path.join(model_path,model_dir_name,'MH49_c_scgen_model_' + str(model_dir_name)+'.pt')
    model = scgen.SCGEN.load(load_string)
    model.adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    model.adata.obs = h.make_annotation_azimut1_5_scanvi_v2(model.adata)
    #if model_dir_name == 'CD4 T':
    #    model.adata = model.adata[(model.adata.obs['scvi_clusters'] != 41)&(model.adata.obs['scvi_clusters'] != 53)]
    model_dict_24hr[model_dir_name] = model
    adata_comp,r2_dict,mean_exp_dict = hscg.similarity_GE_after_shift_24hr(model=model,
                                   adata_train=model.adata,
                                   adata_test=model.adata,
                                   celltype_col= 'cluster_azimut1_5_scanvi_v2',
                                   celltype=model_dir_name)
    adata_comps_24hr[model_dir_name] = adata_comp
    r2_dicts_24hr[model_dir_name] = r2_dict
    mean_exp_dicts_24hr[model_dir_name] = mean_exp_dict
    gene_names_dict_24hr[model_dir_name] = model.adata.var.index.values

In [ ]:
df_test_24hr = pd.DataFrame(index = ['rank_00hr_human', 'rank_06hr_human', 'rank_24hr_human'])
for celltype in celltypes:
    df_ct = hscg.make_rank_df_24hr(mean_exp_dicts_24hr,adata_comps_24hr,celltype=celltype)
    df_test_24hr=df_test_24hr.join(df_ct)

In [ ]:
df_test_24hr.index = ['00hr', '06hr', '24hr']
plt.figure()
sns.heatmap(df_test_24hr.T, annot=True, cmap="PuBuGn", 
            cbar_kws={'label': 'Spearman rank correlation \n predicted human 24hr from cyno 24hr'},
            norm=Normalize(vmin=0.4, vmax=1),
            linecolor='grey',
            linewidths=0.05)
plt.yticks(rotation=0)
plt.savefig(os.path.join(base_plots_path, pre + '_heatmap_rank_24hr.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
for celltype in celltypes:
    df_rank_06hr_ct = h.calculate_df_rank_ct_06hr(mean_exp_dicts_06hr=mean_exp_dicts_06hr,celltype=celltype,
                                                  adata_comps_06hr=adata_comps_06hr)
    df_rank_24hr_ct = h.calculate_df_rank_ct_24hr(mean_exp_dicts_24hr=mean_exp_dicts_24hr,celltype=celltype,
                                                 adata_comps_24hr=adata_comps_24hr)
    for timepoint in ['06hr','24hr']:
        if timepoint == '06hr':
            df_plot,std,mean,upper_bound_99,lower_bound_99,upper_bound_95,lower_bound_95 = h.prepare_bland_altman_plot_06hr_ct(
                df_rank_06hr_ct=df_rank_06hr_ct)
            df_plot.to_csv(os.path.join(base_table_path,'df_bland_altman_' + str(celltype) + '_' + timepoint + '.csv'))
            save=os.path.join(base_plots_path,'plot_bland_altman_' + str(celltype) + '_' + timepoint + '_99.pdf')
            h.do_bland_altman_plot_99(df_plot,lower_bound_99,upper_bound_99,celltype,timepoint,save=save,show=True)

            save=os.path.join(base_plots_path,'plot_bland_altman_' + str(celltype) + '_' + timepoint + '_95.pdf')
            h.do_bland_altman_plot_95(df_plot,lower_bound_95,upper_bound_95,celltype,timepoint,save=save,show=True)
        if timepoint == '24hr':
            df_plot,std,mean,upper_bound_99,lower_bound_99,upper_bound_95,lower_bound_95 = h.prepare_bland_altman_plot_24hr_ct(
                df_rank_24hr_ct=df_rank_24hr_ct)    
            df_plot.to_csv(os.path.join(base_table_path,'df_bland_altman_' + str(celltype) + '_' + timepoint + '.csv'))
            save=os.path.join(base_plots_path,'plot_bland_altman_' + str(celltype) + '_' + timepoint + '_99.pdf')
            h.do_bland_altman_plot_99(df_plot,lower_bound_99,upper_bound_99,celltype,timepoint,save=save,show=True)

            save=os.path.join(base_plots_path,'plot_bland_altman_' + str(celltype) + '_' + timepoint + '_95.pdf')
            h.do_bland_altman_plot_95(df_plot,lower_bound_95,upper_bound_95,celltype,timepoint,save=save,show=True)

### Compare Magnitudes of shifts

In [ ]:
celltypes

In [ ]:
df_shifts = {}
for model_dir_name in celltypes:
    load_string = os.path.join(model_path,model_dir_name,'MH49_c_scgen_model_' + str(model_dir_name)+'.pt')
    model = scgen.SCGEN.load(load_string)
    model.adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    model.adata.obs = h.make_annotation_azimut1_5_scanvi_v2(model.adata)
    latent_adata = hscg.get_latent_representation_object(model=model,adata=model.adata)

    # prepare anndata objects for shifting
    adatas_latent_dict = {}
    for species in all_species:
        for condition in all_conditions:
            adatas_latent_dict[species + '_' +condition] = h.filter_multiple_adata_obs(adata=latent_adata,
                                                            col_names = ['species','timepoint'],
                                                            val_names=[species,condition])

    delta_df_human = hscg.make_delta_df_within_species(adatas_latent_dict=adatas_latent_dict,
                                               species='human',
                                               celltype=model_dir_name,
                                               n_latent=n_latent)

    delta_df_cyno = hscg.make_delta_df_within_species(adatas_latent_dict=adatas_latent_dict,
                                               species='cyno',
                                               celltype=model_dir_name,
                                               n_latent=n_latent)

    shifts_within_species = ['00hr->06hr','00hr->24hr','06hr->24hr']

    df_shift = pd.DataFrame(columns = ['shift_human','shift_cyno','euclidean_distance','cosine_similarity'])
    for human_shift in shifts_within_species:
        for cyno_shift in shifts_within_species:
            delta_cyno = delta_df_cyno.loc[human_shift].filter(regex='^latent_dim_').values
            delta_human = delta_df_human.loc[cyno_shift].filter(regex='^latent_dim_').values
            eucl_dist_diff = np.linalg.norm(delta_human - delta_cyno)
            cos_sim = cosine_similarity(delta_cyno[np.newaxis,:],delta_human[np.newaxis,:])
            df_shift = df_shift.append(pd.DataFrame([human_shift,cyno_shift,eucl_dist_diff,cos_sim[0][0]],
                                                    index = ['shift_human','shift_cyno','euclidean_distance','cosine_similarity']).T)
            df_shift.to_csv(os.path.join(base_table_path, pre + '_' +'shift_eucl_dist_' + model_dir_name + '.csv'))
    df_shifts[model_dir_name] = df_shift

In [ ]:
delta_dfs_cyno = {}
delta_dfs_human = {}
for model_dir_name in celltypes:
    load_string = os.path.join(model_path,model_dir_name,'MH49_c_scgen_model_' + str(model_dir_name)+'.pt')
    model = scgen.SCGEN.load(load_string)
    model.adata.obs.rename(columns={'species_x': 'species'}, inplace=True)
    model.adata.obs = h.make_annotation_azimut1_5_scanvi_v2(model.adata)
    latent_adata = hscg.get_latent_representation_object(model=model,adata=model.adata)


    # prepare anndata objects for shifting
    adatas_latent_dict = {}
    for species in all_species:
        for condition in all_conditions:
            adatas_latent_dict[species + '_' +condition] = h.filter_multiple_adata_obs(adata=latent_adata,
                                                            col_names = ['species','timepoint'],
                                                            val_names=[species,condition])

    delta_df_human = hscg.make_delta_df_within_species(adatas_latent_dict=adatas_latent_dict,
                                               species='human',
                                               celltype=model_dir_name,
                                               n_latent=n_latent)

    delta_df_cyno = hscg.make_delta_df_within_species(adatas_latent_dict=adatas_latent_dict,
                                               species='cyno',
                                               celltype=model_dir_name,
                                               n_latent=n_latent)

    delta_dfs_cyno[model_dir_name] = delta_df_cyno
    delta_dfs_human[model_dir_name] = delta_df_human

In [ ]:
species = 'cyno'
df_magnitude_cyno = pd.DataFrame(columns = ['shift','species','celltype','magnitude','factor_00_06'])
for celltype in celltypes:
    delta_df_cyno = delta_dfs_cyno[celltype]
    for shift in ['00hr->06hr','00hr->24hr']:
        mag = np.linalg.norm(delta_df_cyno.loc[shift].filter(regex='^latent_dim_').values)
        if shift == '00hr->06hr':
            factor_temp = mag.copy()
            df_magnitude_cyno = df_magnitude_cyno.append(pd.DataFrame([shift,species,celltype,mag,1],
                                                            index = ['shift','species','celltype','magnitude','factor_00_06']).T)
        if shift == '00hr->24hr':
            factor = mag/factor_temp
            df_magnitude_cyno = df_magnitude_cyno.append(pd.DataFrame([shift,species,celltype,mag,factor],
                                                            index = ['shift','species','celltype','magnitude','factor_00_06']).T)

In [ ]:
species = 'human'
df_magnitude_human = pd.DataFrame(columns = ['shift','species','celltype','magnitude','factor_00_06'])
for celltype in celltypes:
    delta_df_human = delta_dfs_human[celltype]
    for shift in ['00hr->06hr','00hr->24hr']:
        mag = np.linalg.norm(delta_df_human.loc[shift].filter(regex='^latent_dim_').values)
        if shift == '00hr->06hr':
            factor_temp = mag.copy()
            df_magnitude_human = df_magnitude_human.append(pd.DataFrame([shift,species,celltype,mag,1],
                                                            index = ['shift','species','celltype','magnitude','factor_00_06']).T)
        if shift == '00hr->24hr':
            factor = mag/factor_temp
            df_magnitude_human = df_magnitude_human.append(pd.DataFrame([shift,species,celltype,mag,factor],
                                                            index = ['shift','species','celltype','magnitude','factor_00_06']).T)

In [ ]:
df_magnitude = df_magnitude_cyno.append(df_magnitude_human)
df_magnitude['shift+species'] = df_magnitude['shift'] + str('_') + df_magnitude['species']  

df_magnitude.to_csv(os.path.join(base_table_path,pre + '_df_magnitude_latent.csv'))

In [ ]:
df_magnitude['magnitude'] = pd.to_numeric(df_magnitude['magnitude'], errors='coerce')

df_pivot_cyno = df_magnitude.pivot('shift+species', 'celltype', 'magnitude').T

# Plot heatmap
plt.figure()
sns.heatmap(df_pivot_cyno, annot=True, cmap='Oranges',fmt='.4f', linewidths=0.05,linecolor='grey')
plt.title('Magnitude shift in latent space')
plt.ylabel('Celltype')
plt.xlabel('Shift')
plt.yticks(rotation=0)
plt.savefig(os.path.join(base_plots_path,pre +'heatmap_magnitude_latent_plot.pdf'),bbox_inches='tight')
plt.show()

Cosine similarity between shift magnitude of shifts

In [ ]:
df_shift_cos = pd.DataFrame(index = ['00hr->06hr','00hr->24hr','06hr->24hr'])
for celltype in celltypes:
    cos_00hr_06hr = df_shifts[celltype][(df_shifts[celltype]['shift_human'] ==  '00hr->06hr')
                        & (df_shifts[celltype]['shift_cyno'] ==  '00hr->06hr')]['cosine_similarity'][0] 

    cos_00hr_24hr = df_shifts[celltype][(df_shifts[celltype]['shift_human'] ==  '00hr->24hr')
                        & (df_shifts[celltype]['shift_cyno'] ==  '00hr->24hr')]['cosine_similarity'][0] 

    cos_06hr_24hr = df_shifts[celltype][(df_shifts[celltype]['shift_human'] ==  '06hr->24hr')
                        & (df_shifts[celltype]['shift_cyno'] ==  '06hr->24hr')]['cosine_similarity'][0] 

    df_shift_cos = df_shift_cos.join(pd.DataFrame([cos_00hr_06hr,cos_00hr_24hr,cos_06hr_24hr],index = ['00hr->06hr','00hr->24hr','06hr->24hr'],columns=[celltype]))
df_shift_cos.to_csv(os.path.join(base_table_path,pre + 'df_cos.csv'))

In [ ]:
df_shift_eucl = pd.DataFrame(index = ['00hr->06hr','00hr->24hr','06hr->24hr'])
for celltype in celltypes:
    eucl_00hr_06hr = df_shifts[celltype][(df_shifts[celltype]['shift_human'] ==  '00hr->06hr')
                        & (df_shifts[celltype]['shift_cyno'] ==  '00hr->06hr')]['euclidean_distance'][0] 

    eucl_00hr_24hr = df_shifts[celltype][(df_shifts[celltype]['shift_human'] ==  '00hr->24hr')
                        & (df_shifts[celltype]['shift_cyno'] ==  '00hr->24hr')]['euclidean_distance'][0] 

    eucl_06hr_24hr = df_shifts[celltype][(df_shifts[celltype]['shift_human'] ==  '06hr->24hr')
                        & (df_shifts[celltype]['shift_cyno'] ==  '06hr->24hr')]['euclidean_distance'][0] 

    df_shift_eucl = df_shift_eucl.join(pd.DataFrame([eucl_00hr_06hr,eucl_00hr_24hr,eucl_06hr_24hr],index = ['00hr->06hr','00hr->24hr','06hr->24hr'],columns=[celltype]))
    
df_shift_eucl.to_csv(os.path.join(base_table_path,pre + 'df_eucl.csv'))

In [ ]:
plt.figure()
sns.heatmap(df_shift_cos.T, annot=True,fmt='.4f', cmap="Blues", 
            cbar_kws={'label': 'cosine similarity \nbetween latent shift vectors'},
            linecolor='grey',norm=Normalize(vmin=0, vmax=1),
            linewidths=0.05)
plt.yticks(rotation=0)
plt.savefig(os.path.join(base_plots_path, pre + '_heatmap_cosine_sim_shifts.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
plt.figure()
sns.heatmap(df_shift_eucl.T, annot=True,fmt='.4f', cmap="Purples_r", 
            cbar_kws={'label': 'Euclidean distance \nbetween latent shift vectors'},
            linecolor='grey',
            linewidths=0.05)
plt.yticks(rotation=0)
plt.savefig(os.path.join(base_plots_path, pre + '_heatmap_euclidean_shifts.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()